In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
from sklearn.metrics import accuracy_score
import operator
from timeit import default_timer as timer
from src.main import convert_to_one_hot
from src.main import prepare_attribut_data, prepare_result_data
from src.main import DRAW, HOME_WINS, AWAY_WINS
from sklearn.model_selection import train_test_split


In [2]:

train_path_away = '../data/Train_Data/train_away_team_statistics_df.csv'
train_path_home = '../data/Train_Data/train_home_team_statistics_df.csv'
train_path_result = '../data/Train_Data/Y_train.csv'
df_attribut = prepare_attribut_data(train_path_away, train_path_home).drop('ID',axis=1)
df_result = prepare_result_data(train_path_result)

In [17]:
df_attribut.head()
df_result.head()

,ID,HOME_WINS,DRAW,AWAY_WINS
0,0,0,0,1
1,1,0,1,0
2,2,0,0,1
3,3,1,0,0
4,4,0,1,0


In [ ]:

attribut_train, attribut_test, result_train, result_test = train_test_split(df_attribut, df_result, test_size=0.1,
                                                                            random_state=10)


In [ ]:
result_test

In [ ]:

min_index=0
max_index=20000
for i in [1,2]:   

    column_1=attribut_test.columns[i]
    column_2=attribut_test.columns[i+140]
    plt.figure(figsize=(16, 10))
    plt.scatter(attribut_test.loc[(result_test == DRAW).values, column_1].iloc[min_index:max_index], attribut_test.loc[(result_test == DRAW).values, column_2].iloc[min_index:max_index], label="DRAW", color='gray')

    plt.xlabel(column_1)
    plt.ylabel(column_2)
    plt.title("Train Set")
    plt.legend()
    plt.show()

In [ ]:
from sklearn.linear_model import LogisticRegression

from sklearn.linear_model import LogisticRegression

max_iter_list = [1500]
solver_list = ['lbfgs']  # sag and saga take a lot of time
regularization_factor_list = [ 0.1, 0.0005]
accuracy_dict = {}
time_dict = {}
LR_dict = {}
for max_iter in max_iter_list:
    for solver in solver_list:
        for regularization_factor in regularization_factor_list:
            start = timer()

            LR = LogisticRegression(max_iter=max_iter, solver=solver, C=regularization_factor)
            LR.fit(attribut_train,result_train)
            yhat = LR.predict(attribut_test)
            accuracy = np.round(accuracy_score(result_test, yhat), 4)
            key = f'solver_{solver}_iter_{max_iter}_C_{regularization_factor}'
            accuracy_dict[key] = accuracy
            LR_dict[key] = LR

            end = timer()
            print(f'for this hyperparam {key} the training took ')

            delta = end - start
            minutes = round(delta // 60)
            delta %= 60
            seconds = round(delta)
            time_str = f"{minutes:02d}m{seconds:02d}s"
            time_dict[key] = time_str
            print(time_str)



In [ ]:
accuracy_dict

In [ ]:
max_accuracy = max(accuracy_dict.items(), key=operator.itemgetter(1))
LR_best = LR_dict[max_accuracy[0]]

In [ ]:
time_dict

In [ ]:
max_accuracy

In [ ]:
yhat = LR_best.predict(attribut_test)
np.round(accuracy_score(result_test, yhat), 4)

In [ ]:
yhat

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(result_test, yhat)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=[-2, -1, 1], )
disp.plot(cmap="Blues")

In [ ]:
test_path_away = '../data/Test_Data/test_away_team_statistics_df.csv'
test_path_home = '../data/Test_Data/test_home_team_statistics_df.csv'
df_attribut_test = prepare_attribut_data(test_path_away, test_path_home,is_test=True)

In [ ]:
df_attribut_test

In [ ]:
yhat_test = LR_best.predict(df_attribut_test)

In [ ]:
y_pred_test = convert_to_one_hot(yhat_test)

In [ ]:
test_prediction = pd.concat([df_attribut_test['ID'], y_pred_test], axis=1)

In [ ]:
test_prediction.to_csv('../submission/logistic_benchmark_submission.csv', index=False)